In [1]:
import requests as r
from bs4 import BeautifulSoup
import re
from collections import Counter
import pandas as pd

In [2]:
with open('./lan.text') as f:
    lan_text = f.read().split('\n')
lan_text

['C++',
 'Java',
 'C#',
 'Word',
 'Excel',
 'PowerPoint',
 'Outlook',
 'MS',
 'SQL',
 'Delphi',
 'C',
 'JavaScript',
 'HTML',
 'ASP.NET',
 'Oracle',
 'Visual',
 'Basic',
 '.net',
 'Visual',
 'C#',
 'JSP',
 'HTTP',
 'AutoCAD',
 'Project',
 'Database',
 'Management',
 'SolidWorks',
 'CAM',
 'ARM',
 'Internet',
 'Explorer',
 '鼎新',
 'Linux',
 'jQuery',
 'MySQL',
 'Android',
 'PHP',
 'Visual',
 'C++',
 'Visual',
 'Studio',
 '.net',
 'AJAX',
 'Objective-C',
 'Python',
 'C++.Net',
 'XML',
 'iOS',
 'Visual',
 'Studio',
 'Visual',
 'Basic',
 'Windows',
 '7',
 'TCP/IP',
 'Spring',
 'J2EE',
 'DHTML',
 'Windows',
 'Server',
 '2008',
 'OOP',
 'MFC',
 'Windows',
 'XP',
 'Struts',
 'PL/SQL',
 'Windows',
 'Server',
 '2012',
 'Tomcat',
 'Windows',
 '2000',
 'JDBC',
 'LabVIEW',
 'ASP',
 'SWIFT',
 'Mac',
 'OS',
 'WinForm',
 'Windows',
 '8',
 'UNIX',
 'VBScript',
 'PLC',
 'Windows',
 'NT',
 'UML',
 'XML',
 'Web',
 'services',
 'Servlets',
 'Perl',
 'PostgreSQL',
 'Socket',
 '中文打字20~50',
 'MCU',
 'Windows'

In [9]:
lc = Counter()
for i in range(1,300):
    try:
        Host = 'https://www.1111.com.tw'
        url = Host+'/job-bank/job-index.asp?si=1&ss=s&ks=%E8%BB%9F%E9%AB%94%E5%B7%A5%E7%A8%8B%E5%B8%AB&page={}.format(i)'
        res = r.get(url)
        soup = BeautifulSoup(res.text,'lxml')
        rows = soup.select('div.jbInfo > div > h3 > a')
        
        for row in rows:
            url2 = 'https:'+row['href']
            res2 = r.get(url2)
            soup2 = BeautifulSoup(res2.text,'lxml')
            
            content = str(soup2.select('dl > dd'))
            content2 = str(soup2.select('dl > dd > p'))
            
            word = re.findall('[A-Za-z]+[+#?]*' , content)
            word2 = re.findall('[A-Za-z]+[+#?]*' , content2)
            
            word_count = word.extend(word2)
            
            pure_word = list(set(word_count))
            
            for ind in pure_word:
                if ind in lan_text:
                    if ind in lc:
                        lc[ind] += 1
                    else:
                        lc[ind] = 1
    except:
        continue
    print('[Debug] Page {} is finished'.format(i))
lc.most_common()

[]

In [4]:
#test
url2 = 'https://www.1111.com.tw/job/79817160/'
res2 = r.get(url2)
soup2 = BeautifulSoup(res2.text,'lxml')
soup2.select('dl > dd > p')
#midblock > div.section.w570 > dl > dd:nth-child(2) > p

[<p>1. Python 開發UI介面<br/>2. 熟Qt designer佳<br/>3. 演算法開發<br/>4. 其它主管交辨事項</p>]

In [5]:
df = pd.DataFrame(lc.most_common())
lan_csv = df.to_csv('eeee_lan.csv')
df.head(10)

""


In [6]:
url2

'https://www.1111.com.tw/job/79817160/'

In [7]:
import re
res2 = r.get('https://www.1111.com.tw/job/79850507/')
soup2 = BeautifulSoup(res2.text,'lxml')
content = str(soup2.select('dl > dd'))
lan = re.findall('[A-Za-z]+[+#?]*' , content)
lan

['dd',
 'p',
 'Position',
 'Overview',
 'br',
 'To',
 'develop',
 'testing',
 'applications',
 'and',
 'processes',
 'as',
 'well',
 'as',
 'to',
 'debug',
 'testing',
 'issues',
 'br',
 'Apply',
 'various',
 'software',
 'technology',
 'to',
 'implement',
 'manufacturing',
 'engineering',
 'testing',
 'software',
 'or',
 'tools',
 'br',
 'Able',
 'to',
 'analyze',
 'test',
 'data',
 'to',
 'optimize',
 'test',
 'efficiency',
 'for',
 'production',
 'br',
 'br',
 'Skill',
 'Requirements',
 'br',
 'years',
 'work',
 'experience',
 'for',
 'Windows',
 'desktop',
 'application',
 'development',
 'br',
 'Familiar',
 'with',
 'C++',
 'OOP',
 'Windows',
 'programming',
 'br',
 'Experienced',
 'in',
 'SQL',
 'p',
 'dd',
 'dd',
 'a',
 'ad',
 'id',
 'class',
 'fancybox',
 'fancybox',
 'iframe',
 'href',
 'includesU',
 'locateMap',
 'asp?',
 'ono',
 'amp',
 'eno',
 'amp',
 'xpoint',
 'amp',
 'ypoint',
 'amp',
 'addr',
 'id',
 'showmap',
 'onclick',
 'setHref',
 'img',
 'align',
 'absmiddle',
 'b

In [8]:
#midblock > div:nth-child(2) > dl            .section > dl
#jobResult > ul > li:nth-child(1) > div.jbInfo > div > h3 > a
#midblock > div:nth-child(2) > dl > dd:nth-child(10)